In [1]:
%matplotlib inline

#- import required python libraries
import netCDF4
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import glofrim.glofrim_lib as glib
from os.path import join, isfile, abspath, dirname, basename, normpath
from configparser import ConfigParser
from glofrim.glofrim_lib import configread

In [2]:
from glofrim import Glofrim, PCR, CMF
cbmi = Glofrim()

In [3]:
#- provide file with local paths to model objects
env_fn = r'../../environment.env'
config = configread(env_fn)
testDataDir = config.get('general', 'test_dir')

In [4]:
#- convert model configs to dictionary
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

OrderedDict([(u'engines',
              OrderedDict([('CMF',
                            u'/home/jannis/Programmes/cama-flood_bmi_v3.6.2/src/libcama.so'),
                           ('DFM',
                            u'/home/jannis/Programmes/DFLOWFM/lib/libdflowfm.so'),
                           ('LFP',
                            u'/home/jannis/Programmes/lisflood-bmi-v5.9/liblisflood.so')])),
             (u'general',
              OrderedDict([('test_dir',
                            u'/home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe')]))])

In [5]:
#- provide file with coupling settings
config_fn = join(testDataDir, 'glofrim_PCR2CMF.ini')
# config_fn = join(testDataDir, 'glofrim_PCR2DFM.ini')
# config_fn = join(testDataDir, 'glofrim_WFL2DFM.ini')
# config_fn = join(testDataDir, 'glofrim_PCR2LFP.ini')
config_fn

u'/home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/glofrim_PCR2CMF.ini'

In [6]:
#- provide locations where to extract simulated discharge
x_obs_coord = 10.89 # set to coords of Neu-Darchau
y_obs_coord = 53.23

In [7]:
#- provide start and end date
t_start = datetime(2000,1,1)
t_end = datetime(2000,1,3)
t_start_test = cbmi.set_start_time(t_start)
try:
    t_start == t_start_test
except:
    sys.exit('start time differ with set_var and get_var')

In [8]:
#- create coupled GLOFRIM model and initialize configs
try:
    cbmi.initialize_config(config_fn, env_fn=env_fn)
except Exception as e:
    print e
    sys.exit('configs could not be initialized')

2018-07-23 17:50:29,835 - PCR - INFO - Config initialized
2018-07-23 17:50:29,852 - CMF - INFO - Config initialized


In [9]:
#- create coupled GLOFRIM model and initialize models
try:
    cbmi.initialize_model()
except Exception as e:
    print e
    sys.exit('models could not be initialized')

2018-07-23 17:50:29,862 - PCR - INFO - Getting rgrid info based on landmask_elbe_30min.map
2018-07-23 17:50:29,865 - PCR - INFO - Getting drainage direction from ldd_elbe_30min.map
2018-07-23 17:50:29,867 - CMF - INFO - Getting Unit-Catchment Grid info based on nextxy.tif
2018-07-23 17:50:29,869 - CMF - INFO - Getting drainage direction from nextxy.tif
2018-07-23 17:50:29,919 - PCR - WARNING - /home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/PCR_Elbe/setup_PCR_30min_Elbe_glofrim.ini file overwritten
2018-07-23 17:50:29,921 - PCR - INFO - Ini file written to /home/jannis/PhD/code/GLOFRIMplusCAMA/model_test_data/test_Elbe/PCR_Elbe/setup_PCR_30min_Elbe_glofrim.ini
2018-07-23 17:50:29,926 pcrglobwb_bmi_v203.configuration INFO Model run started at 2018-07-23 17:50:29.922765
2018-07-23 17:50:29,928 pcrglobwb_bmi_v203.configuration INFO Logging output to /home/jannis/PhD/code/GLOFRIMplusCAMA/experiment_GLOFRIM2/PCR/log/setup_PCR_30min_Elbe.ini_2018-07-23T17:50:29.922765.log
201

In [10]:
#- get corresponding indices for models
obs_ind_PCR = cbmi.index(x_obs_coord, y_obs_coord, 'PCR', in1d=False)
print obs_ind_PCR

[67]


In [11]:
obs_ind_CMF = cbmi.index(x_obs_coord, y_obs_coord, 'CMF', in1d=False)
print obs_ind_CMF

2018-07-23 17:50:38,414 rasterio.env DEBUG Entering env context: <rasterio.env.Env object at 0x7fcf7c36a410>
2018-07-23 17:50:38,415 rasterio.env DEBUG Starting outermost env
2018-07-23 17:50:38,416 rasterio.env DEBUG No GDAL environment exists
2018-07-23 17:50:38,418 rasterio.env DEBUG New GDAL environment <rasterio._env.GDALEnv object at 0x7fcf7c36a310> created
2018-07-23 17:50:38,419 rasterio._env DEBUG Logging error handler pushed.
2018-07-23 17:50:38,421 rasterio._env DEBUG All drivers registered.
2018-07-23 17:50:38,422 rasterio._env DEBUG Started GDALEnv <rasterio._env.GDALEnv object at 0x7fcf7c36a310>.
2018-07-23 17:50:38,423 rasterio.env DEBUG Entered env context: <rasterio.env.Env object at 0x7fcf7c36a410>
2018-07-23 17:50:38,425 rasterio._base DEBUG Got coordinate system
2018-07-23 17:50:38,426 rasterio._base DEBUG Authority key: EPSG, value: 4326
2018-07-23 17:50:38,427 rasterio._base DEBUG Nodata success: 1, Nodata value: -9999.000000
2018-07-23 17:50:38,428 rasterio._base

[255]


In [22]:
while t_start < t_end:
    cbmi.update()
    q_sim_PCR = cbmi.get_value_at_indices(PCR.discharge, obs_ind_PCR) # type object 'PCR' has no attribute 'discharge' ?
    print q_sim_PCR

2018-07-23 18:17:01,510 pcrglobwb_bmi_v203.pcrglobwb INFO reading forcings for time 2000-01-11
2018-07-23 18:17:01,673 pcrglobwb_bmi_v203.pcrglobwb INFO updating model to time 2000-01-11
2018-07-23 18:17:02,341 - CMF - INFO - updated model to datetime 2000-01-12
2018-07-23 18:17:02,341 CMF INFO updated model to datetime 2000-01-12


AttributeError: type object 'PCR' has no attribute 'discharge'

In [ ]:
try:
    cbmi.finalize() # not working
except Exception as e:
    print e
    sys.exit('models could not be finalized')